In [2]:
import sqlite3

conn = sqlite3.connect("law.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS laws (
    id TEXT PRIMARY KEY,
    title TEXT NOT NULL,
    content TEXT,
    parent_id TEXT,
    so_hieu TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS law_refs (
    so_hieu TEXT NOT NULL PRIMARY KEY,
    title TEXT NOT NULL,
    content TEXT
)
""")
conn.commit()

In [6]:
conn.close()

In [ ]:
import win32com.client as win32
import os

def convert_doc_to_docx(input_path, output_path=None):
    word = win32.gencache.EnsureDispatch("Word.Application")
    doc = word.Documents.Open((os.path.abspath(input_path)))

    if output_path is None:
        output_path = os.path.splitext(input_path)[0] + ".docx"

    doc.SaveAs(output_path, FileFormat=16)  # 16 = wdFormatXMLDocument (.docx)
    doc.Close()
    word.Quit()

    return output_path

# Example usage:
convert_doc_to_docx(r"E:\Github\LawAssistant\test\luattrậttự.doc")

In [4]:
import re
import hashlib
from docx import Document
from docx.table import Table
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.text.paragraph import Paragraph

def extract_text_from_docx(doc_path):
    doc = Document(doc_path)
    output_text = []
    for child in doc.element.body.iterchildren():
        if isinstance(child, CT_P):
            para = Paragraph(child, doc)
            text = para.text.strip()
            if text:
                output_text.append(text)
        elif isinstance(child, CT_Tbl):
            table = Table(child, doc)
            for row in table.rows:
                for cell in row.cells:
                    text = cell.text.strip()
                    if text:
                        output_text.append(text)
    return output_text

def extract_so_hieu(text_list):
    pattern = r"\d{1,3}/\d{4}/[A-ZĐ0-9\-]+"
    for text in text_list:
        match = re.search(pattern, text)
        if match:
            return match.group(0)
    return None

def extract_law_reference_info(text_lines):
    """
    Xử lý danh sách các dòng văn bản để trích xuất số hiệu, tựa đề và toàn bộ
    nội dung phần mở đầu (sau "Căn cứ...").
    """
    so_hieu_pattern = re.compile(r'(\d{1,3}/\d{4}/QH\d+)')
    date_pattern = re.compile(r'.*ngày\s+\d{1,2}\s+tháng\s+\d{1,2}\s+năm\s+\d{4}')

    tu_khoa_loai_bo = (
        'QUỐC HỘI',
        'CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM',
        'Độc lập - Tự do - Hạnh phúc',
        '------'
    )

    so_hieu = None
    tua_de_parts = []
    noi_dung_parts = []

    # Biến trạng thái để bắt đầu thu thập nội dung
    dang_thu_thap_noi_dung = False
    stop_keywords = ("Chương", "Điều", "Mục")

    for line in text_lines:
        sub_lines = line.strip().split('\n')

        for sub_line in sub_lines:
            sub_line = sub_line.strip()
            if not sub_line:
                continue

            if sub_line.startswith(stop_keywords):
                break

            # Bật chế độ thu thập nội dung khi gặp "Căn cứ"
            if sub_line.startswith('Căn cứ'):
                dang_thu_thap_noi_dung = True

            # Nếu đang trong chế độ thu thập, thêm dòng vào noi_dung_parts
            if dang_thu_thap_noi_dung:
                noi_dung_parts.append(sub_line)
                continue

            # --- Lọc và thu thập tựa đề (nếu chưa đến phần nội dung) ---
            match_sh = so_hieu_pattern.search(sub_line)
            if match_sh:
                so_hieu = match_sh.group(1)
                continue

            if sub_line.startswith(tu_khoa_loai_bo) or date_pattern.match(sub_line):
                continue

            tua_de_parts.append(sub_line)

        if line.strip().startswith(stop_keywords):
            break

    final_title = ' '.join(tua_de_parts).strip()
    final_content = ' '.join(noi_dung_parts).strip()

    return so_hieu, final_title, final_content

def create_law_ref(cursor, connection, so_hieu, title, content):
    cursor.execute(
        """
        REPLACE INTO law_refs (so_hieu, title, content)
        VALUES (?, ?, ?)
        """, (so_hieu, title, content)
    )
    connection.commit()

def collect_content(output_text, start_index):
    content_lines = []
    i = start_index
    while i < len(output_text):
        next_line = output_text[i].strip()
        if (next_line.startswith("Điều")
            or next_line.startswith("Chương")
            or next_line.startswith("Mục")
            or re.match(r"^\s*(\d+\.\s*|[a-zA-ZđĐ]\)\s*)", next_line)):
            break
        if next_line:
            content_lines.append(next_line)
        i += 1
    return "\n".join(content_lines).strip() or None, i

def clean_content(text):
    if text:
        return " ".join(text.split())
    return None

def generate_id(full_path_title):
    return hashlib.sha256(full_path_title.encode('utf-8')).hexdigest()

def create_law_entry(title, content, parent_id, so_hieu, full_path_title, cursor, connection):
    """
    Tạo ID bằng cách băm full_path_title và chèn vào DB.
    Lưu ý: Bảng 'laws' bây giờ phải có cột 'id' kiểu TEXT làm khóa chính.
    """
    entry_id = generate_id(full_path_title)
    cursor.execute(
        "REPLACE INTO laws (id, title, content, parent_id, so_hieu) VALUES (?, ?, ?, ?, ?)",
        (entry_id, title, content, parent_id, so_hieu)
    )
    connection.commit()
    return entry_id

def parse_document(output_text, cursor, conn):
    index = 0
    so_hieu, law_title, law_content = extract_law_reference_info(output_text)
    create_law_ref(cursor, conn, so_hieu, law_title, law_content)
    if not so_hieu:
        raise ValueError("Không tìm thấy số hiệu văn bản.")

    # Biến để lưu trữ ID và tiêu đề của các mục cha hiện tại
    chuong_id = muc_id = dieu_id = khoan_id = None
    chuong_title = muc_title = dieu_title = khoan_title = ""

    while index < len(output_text):
        line = output_text[index].strip()

        # ---- Chương ----
        if line.startswith("Chương"):
            parts = line.split("\n", 1)
            title = parts[0].strip()
            content, index = collect_content(output_text, index + 1)
            content = clean_content(content)

            full_path = f"{so_hieu}_{title}"
            chuong_id = create_law_entry(title, content, None, so_hieu, full_path, cursor, conn)

            # Cập nhật đường dẫn hiện tại và reset các cấp con
            chuong_title = title
            muc_id = dieu_id = khoan_id = None
            muc_title = dieu_title = khoan_title = ""
            continue

        # ---- Mục ----
        if line.startswith("Mục"):
            parts = line.split("\n", 1)
            title = parts[0].strip()
            content, index = collect_content(output_text, index + 1)
            content = clean_content(content)

            full_path = f"{so_hieu}_{chuong_title}_{title}"
            muc_id = create_law_entry(title, content, chuong_id, so_hieu, full_path, cursor, conn)

            # Cập nhật đường dẫn hiện tại và reset các cấp con
            muc_title = title
            dieu_id = khoan_id = None
            dieu_title = khoan_title = ""
            continue

        # ---- Điều ----
        if line.startswith("Điều"):
            parts = re.split(r"\.", line, maxsplit=1)
            title = parts[0].strip()
            content = parts[1].strip() if len(parts) > 1 else ""
            extra_content, index = collect_content(output_text, index + 1)
            if extra_content:
                content = f"{content}\n{extra_content}" if content else extra_content
            content = clean_content(content)

            parent_id = muc_id if muc_id else chuong_id
            parent_path = f"{so_hieu}_{chuong_title}"
            if muc_title:
                parent_path += f"_{muc_title}"

            full_path = f"{parent_path}_{title}"
            dieu_id = create_law_entry(title, content, parent_id, so_hieu, full_path, cursor, conn)

            # Cập nhật đường dẫn và reset cấp con
            dieu_title = title
            khoan_id = None
            khoan_title = ""
            continue

        # ---- Khoản ----
        if re.match(r"^\d+\.", line):
            parts = re.split(r"\.", line, maxsplit=1)
            title_num = parts[0].strip()
            title = f"Khoản {title_num}"
            content = parts[1].strip() if len(parts) > 1 else ""
            extra_content, index = collect_content(output_text, index + 1)
            if extra_content:
                content = f"{content}\n{extra_content}" if content else extra_content
            content = clean_content(content)

            parent_path = f"{so_hieu}_{chuong_title}"
            if muc_title: parent_path += f"_{muc_title}"
            parent_path += f"_{dieu_title}"

            full_path = f"{parent_path}_{title}"
            khoan_id = create_law_entry(title, content, dieu_id, so_hieu, full_path, cursor, conn)
            khoan_title = title
            continue

        # ---- Điểm ----
        match = re.match(r"^([^\W\d_])\)", line, re.UNICODE)
        if match:
            letter = match.group(1)
            title = f"Điểm {letter}"
            parts = re.split(r"\)", line, maxsplit=1)
            content = parts[1].strip() if len(parts) > 1 else ""
            extra_content, index = collect_content(output_text, index + 1)
            if extra_content:
                content = f"{content}\n{extra_content}" if content else extra_content
            content = clean_content(content)

            parent_path = f"{so_hieu}_{chuong_title}"
            if muc_title: parent_path += f"_{muc_title}"
            parent_path += f"_{dieu_title}_{khoan_title}"

            full_path = f"{parent_path}_{title}"
            create_law_entry(title, content, khoan_id, so_hieu, full_path, cursor, conn)
            continue

        index += 1

In [19]:
cursor.execute("DELETE FROM laws")
cursor.execute("DELETE FROM law_refs")
conn.commit()

In [6]:
# "Luật-46-2014-QH13.docx" còn nhiều ngoại lệ
doc_path = r"./data/Bộ luật-92-2015-QH13.docx"

output_text = extract_text_from_docx(doc_path)
parse_document(output_text, cursor, conn)